In [2]:
# importing necessary packages
import pandas as pd
import requests
# To be able to pull the API key from the .env, we need to import and run the load_dotenv function from the dotenv module first.
from dotenv import load_dotenv
load_dotenv()
import os
import json
import flickrapi
import time
import sqlalchemy
from sql_functions import get_dataframe
import sql_functions as sf

In [3]:
# get already existing users
df_unique_owner_update = pd.read_csv('data/unique_owner.csv')

/var/folders/n9/f309cdzj1vg84wrrq1h5nskm0000gn/T/ipykernel_12481/3365089610.py:2: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unique_owner_update = pd.read_csv('data/unique_owner.csv')


In [4]:
df_unique_owner_update.shape

(38801, 38)

In [6]:
df_unique_owner_update.head(1)

,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,stat,person_id,person_nsid,...,person_has_free_educational_resources,person_timezone_label,person_timezone_offset,person_timezone_timezone_id,person_timezone_timezone,person_pro_badge,person_expire,person_mbox_sha1sum__content,code,message
0,0,0.0,0.0,0.0,0.0,0.0,0.0,ok,197843899@N04,197843899@N04,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# store users to list 
list_existing_user = []
for x in df_unique_owner_update.person_id:
    list_existing_user.append(x)

In [8]:
len(list_existing_user)

38801

In [9]:
# get users to update
schema = 'capstone_jorittega'
users_to_update = sf.get_dataframe(f'select user_id from {schema}.photo_exif')


In [10]:
new_user = users_to_update.user_id.unique()

In [11]:
new_user

array(['197843899@N04', '26745338@N06', '31331368@N00', ...,
       '198213676@N03', '197915445@N02', '195436421@N08'], dtype=object)

In [12]:
list_new_user = new_user.tolist()

In [14]:
len(list_new_user)

53224

In [15]:
list_update_user = set(list_new_user) - set(list_existing_user)

In [16]:
len(list_update_user)

15111

In [19]:
if len(list_update_user) == len(set(list_update_user)):
    print("No duplicates found.")
else:
    print("Duplicates found.")

No duplicates found.


In [38]:
type(list_update_user)

list

In [37]:
list_update_user = list(list_update_user)

# calling the API for updated user

In [41]:
df_update_user = pd.DataFrame([])

In [52]:
# run code for updates
url= 'https://www.flickr.com/services/rest/?'
method = 'method=flickr.people.getInfo'
api_key = '&api_key=' + os.getenv('flickr_api_key')
user_id = '&user_id=' 

format = '&format=' + 'json'
nojsoncallback = '&nojsoncallback='+ '1'

counter = 0

try:
    for user in list_update_user[13680:]:
        url_f = url + method + api_key + user_id + user + format + nojsoncallback
        r = requests.get(url_f)
        user_temp = r.json()
        user_temp_df = pd.json_normalize(user_temp, sep="_")

        df_update_user = pd.concat([df_update_user,user_temp_df], ignore_index=True)
        counter += 1
        time.sleep(1)
        print (counter)
        print(df_update_user.shape) 

except flickrapi.exceptions.FlickrError as ex:
        # If accessing the API fails, print something out
        print(f'!!! Error code: {ex.code}')

1
(13681, 31)
2
(13682, 31)
3
(13683, 31)
4
(13684, 31)
5
(13685, 31)
6
(13686, 31)
7
(13687, 31)
8
(13688, 31)
9
(13689, 31)
10
(13690, 31)
11
(13691, 31)
12
(13692, 31)
13
(13693, 31)
14
(13694, 31)
15
(13695, 31)
16
(13696, 31)
17
(13697, 31)
18
(13698, 31)
19
(13699, 31)
20
(13700, 31)
21
(13701, 31)
22
(13702, 31)
23
(13703, 31)
24
(13704, 31)
25
(13705, 31)
26
(13706, 31)
27
(13707, 31)
28
(13708, 31)
29
(13709, 31)
30
(13710, 31)
31
(13711, 31)
32
(13712, 31)
33
(13713, 31)
34
(13714, 31)
35
(13715, 31)
36
(13716, 31)
37
(13717, 31)
38
(13718, 31)
39
(13719, 31)
40
(13720, 31)
41
(13721, 31)
42
(13722, 31)
43
(13723, 31)
44
(13724, 31)
45
(13725, 31)
46
(13726, 31)
47
(13727, 31)
48
(13728, 31)
49
(13729, 31)
50
(13730, 31)
51
(13731, 31)
52
(13732, 31)
53
(13733, 31)
54
(13734, 31)
55
(13735, 31)
56
(13736, 31)
57
(13737, 31)
58
(13738, 31)
59
(13739, 31)
60
(13740, 31)
61
(13741, 31)
62
(13742, 31)
63
(13743, 31)
64
(13744, 31)
65
(13745, 31)
66
(13746, 31)
67
(13747, 31)
68
(

In [51]:
df_update_user.to_csv('data/update_user.csv')